In [8]:
import requests
import json
import time

In [9]:
with open('C:/Users/Dominika/Downloads/high supp limit confidence.json', 'r') as file:
    bodyTemplate = json.load(file)

In [8]:
lines = []
with open("top40_old.csv") as file:
    for line in file: 
        line = line.strip() #or some other preprocessing
        lines.append(line) #storing everything in memory!

In [10]:
import pandas as pd

df = pd.read_csv('top40_old copy.csv', header=None)

In [7]:
bodyTemplate[3]["parameters"]["min"]

100

In [12]:
row_id = 0

for row in df.itertuples():
    row_id+=1

    bodyTemplate[2]["parameters"]["patterns"][0]["head"]["object"]["value"] = "<"+row[1]+">"
    bodyTemplate[2]["parameters"]["thresholds"][0]["value"] = int(row[2]*0.05)
    bodyTemplate[3]["parameters"]["min"] = int(row[2]*0.05)

    URL = "http://rdfrules.vse.cz/api/task"

    body = bodyTemplate

    r = requests.post(url = URL, json = body)

    data = r.json()

    id = data['id']
    print("Odeslal se pozadavek na nalezeni pravidel k mediu ", row[1], ", task id je ", id, "support je ", int(row[2]*0.05))
    print(bodyTemplate)

    #save ids
    f = open("C:/Users/Dominika/Documents/SKOLA/ING/DIPLOMKA/output/ids.txt", "a")
    f.write(str(id)+"\n")
    f.close()

    
    #get requests
    getURL = "http://rdfrules.vse.cz/api/task/"+id


    time.sleep(15)

    r = requests.get(getURL)

    while r.status_code == 202:
        r = requests.get(getURL)
        print(row[1], "(", row_id, "/", len(df), ")", r.status_code," ... LOGY:", r.json()['logs'])
        time.sleep(15)
    else:
        if r.status_code == 200:
            print("\n počet nalezených pravidel k mediu ", row[1], "(", row_id, "/", len(df), ")", ": ", len(r.json()['result']), "\n")
        elif r.status_code != 202 and r.status_code != 200:
            print("\n v mediu ", row[1], " task ", id, " nastala chyba ", r.status_code, "\n")

    for row in r.json()['result']:
        f = open("C:/Users/Dominika/Documents/SKOLA/ING/DIPLOMKA/output/ruleset_lowersup.ndjson", "a")
        f.write(str(row)+"\n")
        f.close()




Odeslal se pozadavek na nalezeni pravidel k mediu  urn:unknown:medium%3A65 , task id je  2e30d132-5f38-48c2-a8e0-e54dfa3b0e78 support je  89
[{'name': 'LoadGraph', 'parameters': {'path': '/kg-microbe/train_data.ttl'}}, {'name': 'Index', 'parameters': {'train': [], 'test': []}}, {'name': 'Mine', 'parameters': {'thresholds': [{'name': 'MinSupport', 'value': 89}, {'name': 'MinHeadSize', 'value': 50}, {'name': 'MaxRuleLength', 'value': 3}, {'name': 'LocalTimeout', 'value': 10000, 'me': 0.2, 'dme': False}], 'ruleConsumers': [{'name': 'topK', 'k': 100000, 'allowOverflow': True}], 'patterns': [{'body': [], 'head': {'subject': {'name': 'Variable', 'value': 'a'}, 'predicate': {'name': 'Constant', 'value': 'biolink:occurs_in'}, 'object': {'name': 'Constant', 'value': '<urn:unknown:medium%3A65>'}, 'graph': {'name': 'Any'}}, 'exact': False, 'orderless': True}], 'constraints': [{'name': 'WithoutPredicates', 'values': ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>', 'rdfs:label']}, {'name': 'On

In [88]:
rules = []
with open("C:/Users/Dominika/Documents/SKOLA/ING/DIPLOMKA/output/ruleset.ndjson") as file:
    for line in file: 
        line = line.strip() #or some other preprocessing
        rules.append(line) #storing everything in memory!

In [90]:
len(rules)

548

In [96]:
for rule in rules:
    objects = json.loads(rule)["body"]
    for object in objects:
        if (isinstance(objects[objects.index(object)]["object"]["value"], dict)):
            if (objects[objects.index(object)]["object"]["value"]["localName"] == "OrganismTaxon"):
                rules.pop(rules.index(rule))

len(rules)

218

In [98]:
for row in rules:
    f = open("C:/Users/Dominika/Documents/SKOLA/ING/DIPLOMKA/output/ruleset_only_good.ndjson", "a")
    f.write(str(row)+"\n")
    f.close()